In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from keras.datasets import mnist
import tensorflow.keras as kb
from tensorflow.keras import backend
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer


from plotnine import *

from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score

from sklearn.linear_model import LinearRegression # Linear Regression Model
from sklearn.preprocessing import StandardScaler #Z-score variables

from sklearn.model_selection import train_test_split # simple TT split cv
from sklearn.model_selection import KFold # k-fold cv
from sklearn.model_selection import LeaveOneOut #LOO cv


# Optimizers

In the lecture we talked about different methods for optimizing our loss function

- **Gradient Descent**
- Gradient Descent with **Momentum**
- **AdaGrad**
- **RMSP**
- **Adam**

All of these are (or are based off of) the basic updating rule from Gradient Desent:

$$ w_t = w_{t-1} - \alpha * g_t $$

Which says that the new weights ($w_t$) are the old weights ($w_{t-1}$) minus some adjustment which is the product of the learning rate ($\alpha$) and the gradient ($g_t$). 


**Momentum** affects the gradient part of the update rule. Rather than updating based on just the current gradient, we update based on the *moving average* of past gradients. This allows us to "build up" speed as we "roll" down the gradient, but also smooths out any osscilating steps that we might take. 

$$ w_t = w_{t-1} - \alpha * m_t $$
$$ m_t = \beta * m_{t-1} + (1 - \beta) * g_t$$

**AdaGrad** and **RMSP** both affect the learning rate part of the update rule. Both optimizers allow us to

1. use different learning rates for different weights/parameters
2. **ada**pt the learning rate throughout the process

AdaGrad:
$$ w_t = w_{t-1} - \frac{\alpha}{\sqrt{\epsilon + \sum g_t^2}} * g_t $$

RMSP:
$$ w_t = w_{t-1} - \frac{\alpha}{\sqrt{\epsilon + \nu_t}} * g_t $$
$$ \nu_t = \beta * \nu_{t-1} + (1 - \beta) * g_t^2$$
AdaGrad updates the learning rate based on the sum of the squared past gradients, whereas RMSP updates the learning rate based on the moving average of the past squared gradients. 

**Adam** combines the changes Momentum makes to the gradient part of the update rule, and the changes RMSP makes to the learning rate. It also unbiases the momentum and learning rate parameters so that they are not as strongly affected by the fact that we initialize the past gradients to be 0. 

$$ w_t = w_{t-1} - \frac{\alpha}{\sqrt{\epsilon + \hat{\nu_t}}} * \hat{m_t} $$
$$ \nu_t = \beta_2 * \nu_{t-1} + (1 - \beta_2) * g_t^2$$
$$m_t = \beta_1 * m_{t-1} + (1 - \beta_1) * g_t$$
$$ \hat{\nu_t} = \frac{\nu_t}{1 - \beta_2^t},\; \hat{m_t} = \frac{m_t}{1 - \beta_1^t}$$

## Using Different Optimizers in Keras/Tensorflow


## Previously

In a past lecture, we looked at a python implementation of Gradient Descent for a simple linear regression model using a Sum of Square Errors loss function. The function below, `stepGradient()` takes in four arguments:
- `b0_current`: the current value for the `b0` intercept parameter
- `b1_current`: the current value for the `b1` slope parameter
- `point` a DataFrame of the points we're using to claculate the gradient
- `learningRate` a constant value representing the learning rate (how big of a step we should take at each step)


### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200"/>

Take a moment to study this function and map it to the process that we learned about for gradient descent. Call this function using `b0_current` = 0, `b1_current` = 0, `points` = `df`, and `learningRate` = 0.1

In [2]:
def stepGradient(b0_current, b1_current, points, learningRate):
    # initialize gradient to 0
    b0_gradient = 0
    b1_gradient = 0

    # for each data point, calculate gradient and add 
    for i in range(0, len(points)):
        b0_gradient += -2 * (points.iloc[i].y - ((b1_current*points.iloc[i].x) + b0_current))
        b1_gradient += -2 * points.iloc[i].x * (points.iloc[i].y - ((b1_current * points.iloc[i].x) + b0_current))

    b0_gradient = b0_gradient/points.shape[0]
    b1_gradient = b1_gradient/points.shape[0]
    # update parameter values
    new_b0 = b0_current - (learningRate * b0_gradient)
    new_b1 = b1_current - (learningRate * b1_gradient)
    return [np.round(new_b0,5), np.round(new_b1,5)]

#based on https://spin.atomicobject.com/2014/06/24/gradient-descent-linear-regression/

In [4]:
# create data frame 
np.random.seed(1234)
x = np.random.normal(loc = 0, scale = 1, size = 100)
y = 1.67 + 0.67*x + np.random.normal(loc = 0, scale = 0.2, size = 100)
df = pd.DataFrame({"x": x, "y": y})
df

,x,y
0,0.471435,2.044103
1,-1.190976,0.985353
2,1.432707,2.730632
3,-0.312652,1.517582
4,-0.720589,1.284063
...,...,...
95,-0.081947,1.528786
96,-0.344766,1.406779
97,0.528288,2.201785
98,-1.068989,1.011453


In [6]:
# GD
a = 0
b = 0

# run 500 updates
for i in range(0,500):
  a,b = stepGradient(a,b, df, 0.1)

  # every 10 updates, print the current parameter values
  if i%10 == 0:
    print(a,b)

0.3367 0.1479
1.522 0.63978
1.64503 0.68249
1.65789 0.686
1.65923 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627
1.65938 0.68627


## Momentum
Now that we've learned about momentum, let's modify our `stepGradient()` function to incorporate momentum. 

Remember momentum is calculated based on a decaying sum of the past gradients

$$w_t = w_{t-1} - \alpha * m_t$$
$$m_t = \beta * m_{t-1} + (1-\beta)* g_t$$

This new `stepMomentum()` function will need to take in **three** additional arguments:

- `b0_mt`: $m_t$; the sum of the gradients for `b0` from the previous step
- `b1_mt`: $m_t$; the sum of the gradients for `b1` from the previous step
- `beta`: moving average parameter that controlls how much of the previous gradient is remembered (set to a default value of `0.9`)


### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200"/>

I have added these arguments to the function definition for you. Take a moment to look at the lecture slides and understand the difference between Gradient Descent and Gradient Descent *with* momentum. 

Then, modify the function below (which contains the same code as the `stepGradient()` function) to implement Gradient Descent *with* momentum.

In [8]:
### YOUR CODE HERE ###
# Change this function so it does GD with momentum, right now it just does GD

def stepMomentum(b0_current, b1_current, points, learningRate, b0_mt, b1_mt, beta):
    # initialize gradient to 0
    b0_gradient = 0
    b1_gradient = 0

    # for each data point, calculate gradient and add 
    for i in range(0, len(points)):
        b0_gradient += -2 * (points.iloc[i].y - ((b1_current*points.iloc[i].x) + b0_current))
        b1_gradient += -2 * points.iloc[i].x * (points.iloc[i].y - ((b1_current * points.iloc[i].x) + b0_current))
    
    b0_gradient = b0_gradient/points.shape[0]
    b1_gradient = b1_gradient/points.shape[0]

    # calculate momentum
    b0_mt += (beta*b0_mt) + (1-beta) * b0_gradient
    b1_mt += (beta*b1_mt) + (1-beta) * b1_gradient
    # update parameter values
    new_b0 = b0_current - (learningRate * b0_mt)
    new_b1 = b1_current - (learningRate * b1_mt)
    return [np.round(new_b0,5), np.round(new_b1,5)]

#based on https://spin.atomicobject.com/2014/06/24/gradient-descent-linear-regression/

## Try it Out

Once you've created and tested your `stepMomentum()` function, try it out with this dataset, `df` and compare it to the output of the `stepGradient()` function on the same data.

Initialize `b0_mt` and `b1_mt` both to 0.

### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200"/>

What's different about the updates you got?

In [10]:
### YOUR CODE HERE ###

# GD with Momentum
a = 0
b = 0

# run 500 updates
for i in range(0,500):
  a,b = stepMomentum(a,b, df, 0.1, 0, 0, 0.9)

  # every 10 updates, print the current parameter values
  if i%10 == 0:
    print(a,b)

0.03367 0.01479
0.33497 0.14632
0.58038 0.25218
0.78029 0.33737
0.94312 0.40591
1.07579 0.46105
1.18386 0.50538
1.27189 0.54105
1.34361 0.56972
1.40205 0.59274
1.44967 0.61125
1.48847 0.62611
1.52008 0.63804
1.54584 0.64763
1.56683 0.65532
1.58395 0.66149
1.59792 0.66645
1.60928 0.67042
1.61853 0.6736
1.62609 0.67614
1.63224 0.67818
1.63729 0.67981
1.64138 0.68111
1.64471 0.68216
1.64742 0.683
1.64963 0.68366
1.65143 0.68419
1.6529 0.68461
1.6541 0.68495
1.65507 0.68523
1.65587 0.68543
1.65652 0.68563
1.65705 0.68573
1.65748 0.68583
1.65783 0.68593
1.65813 0.68603
1.65834 0.68608
1.65854 0.68608
1.6587 0.68608
1.6588 0.68608
1.6589 0.68608
1.659 0.68608
1.6591 0.68608
1.65916 0.68608
1.65916 0.68608
1.65916 0.68608
1.65916 0.68608
1.65916 0.68608
1.65916 0.68608
1.65916 0.68608


### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200"/>

What do you think the potential downside of initializing `b0_mt` and `b1_mt` to `0` is?

## AdaGrad

We also learned about an upgraded version of Gradient Descent called **AdaGrad**. AdaGrad updates the learning rate for each parameter separately by scaling a baseline learning rate, $\alpha$ by the square root of the sum of the previous gradients.

Similarly to what we did with momentum, update the code from the `stepGradient()` function below to implement AdaGrad. 

The AdaGrad will need two extra arguments:
- `b0_squared_gradient_sum`: the sum of the previous squared gradients for `b0`
- `b1_squared_gradient_sum`: the sum of the previous squared gradients for `b1`

(Set $\epsilon$ to be 0.0001)

In [18]:
def stepAda(b0_current, b1_current, points, learningRate, b0_squared_gradient_sum, b1_squared_gradient_sum ):

    # initialize gradient to 0
    b0_gradient = 0
    b1_gradient = 0

    # for each data point, calculate gradient and add 
    for i in range(0, len(points)):
        b0_gradient += -2 * (points.iloc[i].y - ((b1_current*points.iloc[i].x) + b0_current))
        b1_gradient += -2 * points.iloc[i].x * (points.iloc[i].y - ((b1_current * points.iloc[i].x) + b0_current))

    b0_gradient = b0_gradient/points.shape[0]
    b1_gradient = b1_gradient/points.shape[0]

    b0_squared_gradient_sum += b0_gradient * b0_gradient
    b1_squared_gradient_sum += b1_gradient * b1_gradient

    # square root

    epsilon = 0.0001
    sr_b0 = np.sqrt(epsilon + b0_squared_gradient_sum)
    sr_b1 = np.sqrt(epsilon + b1_squared_gradient_sum)

    # update parameter values
    new_b0 = b0_current - (learningRate/sr_b0) * b0_gradient
    new_b1 = b1_current - (learningRate/sr_b1) * b1_gradient
    return [np.round(new_b0,5), np.round(new_b1,5)]

In [19]:
### YOUR CODE HERE ###

# GD with Momentum
a = 0
b = 0

# run 500 updates
for i in range(0,500):
  a,b = stepAda(a,b, df, 0.1, 0, 0)

  # every 10 updates, print the current parameter values
  if i%10 == 0:
      print(a,b)

0.1 0.1
1.1 0.69271
1.7003 0.66239
1.70201 0.65445
1.70338 0.65002
1.70448 0.64697
1.70536 0.64471
1.70607 0.643
1.70665 0.64166
1.70711 0.64062
1.7075 0.6398
1.7078 0.63914
1.70805 0.63861
1.70825 0.6382
1.70843 0.63787
1.70854 0.6376
1.70864 0.63739
1.70874 0.63723
1.70883 0.63709
1.70888 0.63699
1.70893 0.6369
1.70896 0.63685
1.70896 0.6368
1.70897 0.63675
1.70897 0.6367
1.70897 0.63665
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664
1.70897 0.63664


## RMSProp

We also learned about another algorithm, RMSProp which:

- scales the learning rate by the moving average of the squared past gradients
- tailors the learning rate for each parameter separately



In [20]:
def stepRMSP(b0_current, b1_current, points, learningRate,b0_squared_gradient_sum, b1_squared_gradient_sum, beta ):
    # initialize gradient to 0
    b0_gradient = 0
    b1_gradient = 0

    # for each data point, calculate gradient and add 
    for i in range(0, len(points)):
        b0_gradient += (1/10000) * -2 * (points.iloc[i].y - ((b1_current*points.iloc[i].x) + b0_current))
        b1_gradient += (1/10000) * -2 * points.iloc[i].x * (points.iloc[i].y - ((b1_current * points.iloc[i].x) + b0_current))
    b0_gradient = b0_gradient/points.shape[0]
    b1_gradient = b1_gradient/points.shape[0]
    
    # new learning rates
    b0_squared_gradient_sum = (beta *b0_squared_gradient_sum) + ((1-beta) * b0_gradient**2)
    b1_squared_gradient_sum = (beta *b1_squared_gradient_sum) + ((1-beta) * b1_gradient**2)

    b0_learningRate = learningRate/(np.sqrt(0.0001 + b0_squared_gradient_sum))
    b1_learningRate = learningRate/(np.sqrt(0.0001 + b1_squared_gradient_sum))

    
    # update parameter values
    new_b0 = b0_current - (b0_learningRate * b0_gradient)
    new_b1 = b1_current - (b1_learningRate * b1_gradient)
    return [np.round(new_b0,5), np.round(new_b1,5), b0_squared_gradient_sum, b1_squared_gradient_sum]
#based on https://spin.atomicobject.com/2014/06/24/gradient-descent-linear-regression/

In [26]:
### YOUR CODE HERE ###

# GD with Momentum
a = 0
b = 0

# run 500 updates
for i in range(0,500):
  a,b, b0_squared_gradient_sum, b1_squared_gradient_sum = stepRMSP(a,b, df, 0.1, 0, 0, 0.9)

  # every 10 updates, print the current parameter values
  if i%10 == 0:
      print(a,b)

0.00337 0.00148
0.03667 0.0161
0.0693 0.03041
0.10127 0.04441
0.13259 0.05811
0.16329 0.07152
0.19337 0.08465
0.22284 0.0975
0.25171 0.11008
0.28001 0.12238
0.30774 0.13443
0.33491 0.14623
0.36154 0.15778
0.38764 0.16908
0.41319 0.18013
0.43824 0.19095
0.46279 0.20154
0.48684 0.21191
0.5104 0.22205
0.5335 0.23198
0.55613 0.24169
0.57831 0.2512
0.60003 0.26051
0.62132 0.26962
0.64219 0.27853
0.66263 0.28726
0.68266 0.2958
0.70228 0.30415
0.72152 0.31233
0.74037 0.32033
0.75883 0.32817
0.77693 0.33583
0.79467 0.34333
0.81205 0.35068
0.82907 0.35786
0.84575 0.3649
0.8621 0.37178
0.87812 0.37852
0.89381 0.38511
0.90919 0.39156
0.92426 0.39789
0.93903 0.40406
0.9535 0.41011
0.96769 0.41604
0.98159 0.42184
0.9952 0.42751
1.00854 0.43306
1.02162 0.43849
1.03443 0.44381
1.04698 0.44902


## Adam

Finally, we learned about Adam, which combines Momentum and RMSP.

I won't make you figure this one out on your own, but the code is below!

In [32]:
def stepAdam(b0_current, b1_current, points, learningRate,
             b0_squared_gradient_sum, b1_squared_gradient_sum,
             b0_mt, b1_mt, beta1, beta2,t ):
    # initialize gradient to 0
    b0_gradient = 0
    b1_gradient = 0

    # for each data point, calculate gradient and add 
    for i in range(0, len(points)):
        b0_gradient += (1/10000) * -2 * (points.iloc[i].y - ((b1_current*points.iloc[i].x) + b0_current))
        b1_gradient += (1/10000) * -2 * points.iloc[i].x * (points.iloc[i].y - ((b1_current * points.iloc[i].x) + b0_current))
    
    b0_gradient = b0_gradient/points.shape[0]
    b1_gradient = b1_gradient/points.shape[0]

    # calculate mts
    b0_mt = (beta1*b0_mt) + ((1-beta1)*b0_gradient)
    b1_mt = (beta1*b1_mt) + ((1-beta1)*b1_gradient)

    # unbias these estimates
    b0_mt_unbiased = b0_mt/(1-beta1**t)
    b1_mt_unbiased = b1_mt/(1-beta1**t)

    # new learning rates
    b0_squared_gradient_sum = (beta2 *b0_squared_gradient_sum) + ((1-beta2) * b0_gradient**2)
    b1_squared_gradient_sum = (beta2 *b1_squared_gradient_sum) + ((1-beta2) * b1_gradient**2)

    # unbias these estimates
    b0_squared_gradient_sum_unbiased = b0_squared_gradient_sum/(1-beta2**t)
    b1_squared_gradient_sum_unbiased = b1_squared_gradient_sum/(1-beta2**t)

    b0_learningRate = learningRate/(np.sqrt(0.0001 + b0_squared_gradient_sum_unbiased))
    b1_learningRate = learningRate/(np.sqrt(0.0001 + b1_squared_gradient_sum_unbiased))

    
    # update parameter values
    new_b0 = b0_current - (b0_learningRate * b0_mt_unbiased)
    new_b1 = b1_current - (b1_learningRate * b1_mt_unbiased)
    return [np.round(new_b0,5), np.round(new_b1,5), b0_squared_gradient_sum, b1_squared_gradient_sum, b0_mt, b1_mt]
#based on https://spin.atomicobject.com/2014/06/24/gradient-descent-linear-regression/

## Linear Regression (Comparison)

Just for comparison, since this is a simple model, let's see what Linear Regression comes up with for this model!

### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200"/>

How close did our various methods of Gradient Descent get to Linear Regression's parameter estimates?

In [22]:
from sklearn.linear_model import LinearRegression 
lr = LinearRegression()

lr.fit(df[["x"]], df["y"])
print(lr.intercept_, lr.coef_[0])

1.6593974456567318 0.6862828977798419
